In [ ]:
# to import files from googledrive
from google.colab import drive


drive.mount('/content/gdrive/', force_remount=True)
sample_size_dir = 'gdrive/My Drive/Sample size for NLP'

Mounted at /content/gdrive/


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
!pip install -q transformers datasets scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.


In [ ]:
!pip install -q accelerate -U
!pip install -q transformers[torch]

In [ ]:
## Testing on imdb data for code checks - Ignore if using actual data ##

'''
from datasets import load_dataset

# Load a dataset, for example, the IMDb dataset
dataset = load_dataset('imdb')

### Use only a portion of the data ###

# Define the proportion of the dataset to use
train_size = 0.1
test_size = 0.1t

# Shuffle and select a subset of the dataset
train_dataset = dataset['train'].shuffle(seed=42).select(range(int(train_size * len(dataset['train']))))
test_dataset = dataset['test'].shuffle(seed=42).select(range(int(test_size * len(dataset['test']))))
'''

In [ ]:
df1 = pd.read_csv(sample_size_dir+'/data_Kai/new_6000/4019_merged_report_6000_without.csv')
df2 = pd.read_csv(sample_size_dir+'/data_Kai/new_6000/4019_merged_report_6000_with.csv')

# Randomly sample 500 rows from each DataFrame
sampled_df1 = df1.sample(n=500, random_state=42)
sampled_df2 = df2.sample(n=500, random_state=42)

# Combine the sampled data
combined_df = pd.concat([sampled_df1, sampled_df2])

# Add a column 'label' and set it to 1 if 'ICD9_CODE' is 4019, else 0
combined_df['label'] = combined_df['ICD9_CODE'].apply(lambda x: 1 if x == 4019 else 0)

# Shuffle the combined DataFrame
shuffled_combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Save the combined data to a new CSV file
combined_df.to_csv(sample_size_dir+'/data_jaya/4019_combined_data_1000.csv', index=False)


In [ ]:
# Separate the DataFrame into two DataFrames based on the label
df_label_1 = combined_df[combined_df['label'] == 1]
df_label_0 = combined_df[combined_df['label'] == 0]

# Split each DataFrame into train and test sets
train_label_1, test_label_1 = train_test_split(df_label_1, test_size=0.2, random_state=42)
train_label_0, test_label_0 = train_test_split(df_label_0, test_size=0.2, random_state=42)

# Concatenate the train and test sets
train_dataset = pd.concat([train_label_1, train_label_0]).sample(frac=1, random_state=42).reset_index(drop=True)
test_dataset = pd.concat([test_label_1, test_label_0]).sample(frac=1, random_state=42).reset_index(drop=True)

# Save the subsets to new CSV files
train_dataset.to_csv(sample_size_dir+'/data_jaya/4019_train_dataset.csv', index=False)
test_dataset.to_csv(sample_size_dir+'/data_jaya/4019_test_dataset.csv', index=False)

In [ ]:
train_dataset['label'].value_counts(normalize=True)

label
0    0.5
1    0.5
Name: proportion, dtype: float64

In [ ]:
test_dataset['label'].value_counts(normalize=True)

label
1    0.5
0    0.5
Name: proportion, dtype: float64

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


# Function to tokenize the text
def tokenize_function(text):
    return tokenizer(text, padding="max_length", truncation=True)

# Assuming train_dataset and test_dataset are pandas DataFrames with columns 'TEXT' and 'label'
# Tokenize the datasets
train_dataset['input_ids'] = train_dataset['TEXT'].apply(lambda x: tokenize_function(x)['input_ids'])
train_dataset['attention_mask'] = train_dataset['TEXT'].apply(lambda x: tokenize_function(x)['attention_mask'])

test_dataset['input_ids'] = test_dataset['TEXT'].apply(lambda x: tokenize_function(x)['input_ids'])
test_dataset['attention_mask'] = test_dataset['TEXT'].apply(lambda x: tokenize_function(x)['attention_mask'])

# Rename the label column to 'labels' (assuming 'label' is the original column name)
train_dataset = train_dataset.rename(columns={'label': 'labels'})
test_dataset = test_dataset.rename(columns={'label': 'labels'})

# Convert to lists of dictionaries for Trainer compatibility
train_examples = [{'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels} for input_ids, attention_mask, labels in zip(train_dataset['input_ids'], train_dataset['attention_mask'], train_dataset['labels'])]
test_examples = [{'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels} for input_ids, attention_mask, labels in zip(test_dataset['input_ids'], test_dataset['attention_mask'], test_dataset['labels'])]


'''
# for imbd
#def tokenize_function(examples):
#    return tokenizer(examples['TEXT'], padding="max_length", truncation=True) # 'text' if using imdb

# Define a function to tokenize the text - for dataframe
def tokenize_function(text):
    return tokenizer(text, padding="max_length", truncation=True)

### If using the entire dataset ###

# tokenized_datasets = dataset.map(tokenize_function, batched=True)
# tokenized_datasets = tokenized_datasets.remove_columns(["text"])
# tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
# tokenized_datasets.set_format("torch")

### If using portion of dataset ###

# Tokenize the datasets - for dataframe
train_dataset['input_ids'] = train_dataset['TEXT'].apply(lambda x: tokenize_function(x)['input_ids'])
train_dataset['attention_mask'] = train_dataset['TEXT'].apply(lambda x: tokenize_function(x)['attention_mask'])

test_dataset['input_ids'] = test_dataset['TEXT'].apply(lambda x: tokenize_function(x)['input_ids'])
test_dataset['attention_mask'] = test_dataset['TEXT'].apply(lambda x: tokenize_function(x)['attention_mask'])

# Tokenize the datasets - for huggingface data like imdb
# tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
# tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

# Remove unnecessary columns and set format to PyTorch
#tokenized_train_dataset = train_dataset.drop(["TEXT"]) # 'text' if using imdb and remove_columns instead of drop
#tokenized_test_dataset = test_dataset.drop(["TEXT"]) # 'text' if using imdb and remove_columns instead of drop

tokenized_train_dataset = train_dataset
tokenized_test_dataset = test_dataset
train_dataset = train_dataset.rename(columns={'label': 'labels'})
test_dataset = test_dataset.rename(columns={'label': 'labels'})
#tokenized_train_dataset = tokenized_train_dataset.rename_column("label", "labels") #rename_column if using imdb
#tokenized_test_dataset = tokenized_test_dataset.rename_column("label", "labels") #rename_column if using imdb

# Now train_dataset and test_dataset have columns: 'TEXT', 'input_ids', 'attention_mask', 'labels'

# Convert to PyTorch tensors (assuming labels are already numeric)
train_input_ids = torch.tensor(train_dataset['input_ids'].tolist(), dtype=torch.long)
train_attention_mask = torch.tensor(train_dataset['attention_mask'].tolist(), dtype=torch.long)
train_labels = torch.tensor(train_dataset['labels'].tolist(), dtype=torch.long)

test_input_ids = torch.tensor(test_dataset['input_ids'].tolist(), dtype=torch.long)
test_attention_mask = torch.tensor(test_dataset['attention_mask'].tolist(), dtype=torch.long)
test_labels = torch.tensor(test_dataset['labels'].tolist(), dtype=torch.long)

# Create TensorDatasets
tokenized_train_dataset = TensorDataset(train_input_ids, train_attention_mask, train_labels)
tokenized_test_dataset = TensorDataset(test_input_ids, test_attention_mask, test_labels)


# Example of how to create DataLoader (you can adjust batch_size and shuffle as needed)
#tokenized_test_dataloader = torch.utils.data.DataLoader(tokenized_test_dataset, batch_size=16, shuffle=False)
#tokenized_train_dataloader = torch.utils.data.DataLoader(tokenized_train_dataset, batch_size=16, shuffle=True)


#tokenized_test_dataset.set_format("torch")
#tokenized_train_dataset.set_format("torch")
'''

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


'\n# for imbd\n#def tokenize_function(examples):\n#    return tokenizer(examples[\'TEXT\'], padding="max_length", truncation=True) # \'text\' if using imdb\n\n# Define a function to tokenize the text - for dataframe\ndef tokenize_function(text):\n    return tokenizer(text, padding="max_length", truncation=True)\n\n### If using the entire dataset ###\n\n# tokenized_datasets = dataset.map(tokenize_function, batched=True)\n# tokenized_datasets = tokenized_datasets.remove_columns(["text"])\n# tokenized_datasets = tokenized_datasets.rename_column("label", "labels")\n# tokenized_datasets.set_format("torch")\n\n### If using portion of dataset ###\n\n# Tokenize the datasets - for dataframe\ntrain_dataset[\'input_ids\'] = train_dataset[\'TEXT\'].apply(lambda x: tokenize_function(x)[\'input_ids\'])\ntrain_dataset[\'attention_mask\'] = train_dataset[\'TEXT\'].apply(lambda x: tokenize_function(x)[\'attention_mask\'])\n\ntest_dataset[\'input_ids\'] = test_dataset[\'TEXT\'].apply(lambda x: tokenize_

In [ ]:
# To try truncating from the beginning of the text rather than the end

'''
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define the maximum length for BERT
max_length = 512  # Adjust based on your BERT model

# Function to truncate the beginning of the text
def truncate_text(text, max_length):
    tokens = tokenizer.tokenize(text)
    if len(tokens) > max_length:
        tokens = tokens[-max_length:]
    return tokenizer.convert_tokens_to_string(tokens)

# Define a function to tokenize the text
def tokenize_function(text):
    truncated_text = truncate_text(text, max_length)
    return tokenizer(truncated_text, padding="max_length", truncation=True, max_length=max_length)

# Tokenize the datasets
train_dataset['input_ids'] = train_dataset['TEXT'].apply(lambda x: tokenize_function(x)['input_ids'])
train_dataset['attention_mask'] = train_dataset['TEXT'].apply(lambda x: tokenize_function(x)['attention_mask'])

test_dataset['input_ids'] = test_dataset['TEXT'].apply(lambda x: tokenize_function(x)['input_ids'])
test_dataset['attention_mask'] = test_dataset['TEXT'].apply(lambda x: tokenize_function(x)['attention_mask'])

# Drop the original text column
train_dataset = train_dataset.drop(columns=['TEXT'])
test_dataset = test_dataset.drop(columns=['TEXT'])

# Rename the label column to labels
train_dataset = train_dataset.rename(columns={'label': 'labels'})
test_dataset = test_dataset.rename(columns={'label': 'labels'})

# Define a custom PyTorch Dataset - for compatibility with PyTorch DataLoader so it can be used in batches, and is more memory efficient for parallel loading and processing
class CustomDataset(Dataset):
    def __init__(self, df):
        self.labels = df['labels'].values
        self.input_ids = list(df['input_ids'].values)
        self.attention_mask = list(df['attention_mask'].values)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }
        return item

# Convert the DataFrame to a PyTorch Dataset
tokenized_train_dataset = CustomDataset(train_dataset)
tokenized_test_dataset = CustomDataset(test_dataset)
'''

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

def model_init():
    return BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

In [ ]:
from sklearn.model_selection import ParameterGrid
from transformers import Trainer, TrainingArguments

# Define function to train the model
def train_model(params):
    training_args = TrainingArguments(
        output_dir=sample_size_dir+'/data_jaya/results',
        learning_rate=params['learning_rate'],
        per_device_train_batch_size=params['per_device_train_batch_size'],
        num_train_epochs=params['num_train_epochs'],
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy="epoch"
    )

    trainer = Trainer(
        model_init=model_init,
        args=training_args,
        train_dataset=train_examples,
        eval_dataset=test_examples
    )

    trainer.train()
    eval_result = trainer.evaluate()
    return eval_result['eval_loss']


In [ ]:
output_dir= sample_size_dir+'/data_jaya/results'
output_dir

'gdrive/My Drive/Sample size for NLP/data_jaya/results'

# Grid Search

In [ ]:
# Define the grid of hyperparameters
param_grid = {
    'learning_rate': [5e-5, 3e-5, 2e-5],
    'per_device_train_batch_size': [8, 16],
    'num_train_epochs': [3, 4]
}

# Convert param_grid to a list of parameter dictionaries
param_list = list(ParameterGrid(param_grid))

# List to store results
results = []

# Conduct the grid search
best_loss = float('inf')
best_params = None

for params in param_list:
    eval_loss = train_model(params)
    results.append({'params': params, 'eval_loss': eval_loss})

    if eval_loss < best_loss:
        best_loss = eval_loss
        best_params = params

print("Best params:", best_params)
print("Best loss:", best_loss)

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Save DataFrame to a CSV file
results_df.to_csv(sample_size_dir+'/data_jaya/grid_search_results_53081.csv', index=False)



In [ ]:
print("Best params:", best_params)
print("Best loss:", best_loss)

Best params: {'learning_rate': 5e-05, 'num_train_epochs': 3, 'per_device_train_batch_size': 16}
Best loss: 0.26487576961517334


In [ ]:
results_df = pd.read_csv(sample_size_dir+'/data_jaya/grid_search_results_53081.csv')
results_df.head()

In [ ]:
results_df.to_csv(sample_size_dir+'/data_jaya/grid_search_results.csv', index=False)

# Bayesian Optimisation

Bayesian optimization is a strategy for optimizing black-box functions that are expensive to evaluate. It is particularly useful in scenarios where each function evaluation is costly, such as hyperparameter tuning in machine learning models. It fits a Gaussian Process to model the relationship between hyperparameters and performance. And then uses the Gaussian Process to predict performance and compute the acquisition function to identify the next hyperparameter set to try.

A Gaussian function, also known as a Gaussian distribution or normal distribution, is a symmetric, bell-shaped function that is widely used in statistics, probability theory, and various scientific fields. It describes how values of a variable are distributed, with most values clustering around a central peak and probabilities tapering off as you move away from the center.

In [ ]:
!pip install -q optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.6 MB/s eta 0:00:00


In [ ]:
def model_init():
    return BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

In [ ]:
import optuna

# List to store results
results = []

def objective(trial):
    # Define the hyperparameter search space
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-5)
    per_device_train_batch_size = trial.suggest_categorical('per_device_train_batch_size', [8, 16])
    num_train_epochs = trial.suggest_int('num_train_epochs', 2, 4)

    # Define the training arguments
    training_args = TrainingArguments(
        output_dir='./results',
        learning_rate=learning_rate,
        per_device_train_batch_size=per_device_train_batch_size,
        num_train_epochs=num_train_epochs,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy="epoch"    # evaluate at the end of each epoch
    )

    # Define the trainer
    trainer = Trainer(
        model_init=model_init,
        args=training_args,
        train_dataset=train_examples,
        eval_dataset=test_examples
    )

    # Train the model
    trainer.train()
    eval_result = trainer.evaluate()
    return eval_result['eval_loss']

    # Append trial results to the list
    results.append({
        'learning_rate': learning_rate,
        'per_device_train_batch_size': per_device_train_batch_size,
        'num_train_epochs': num_train_epochs,
        'eval_loss': eval_result['eval_loss']
    })

    return eval_result['eval_loss']


In [ ]:
# Create a study and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

# Get the best hyperparameters
print("Best trial:")
trial = study.best_trial

print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Save DataFrame to a CSV file
results_df.to_csv(sample_size_dir+'/data_jaya/optuna_results_53081.csv', index=False)



In [ ]:
# Get the best hyperparameters
print("Best trial:")
trial = study.best_trial

print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

Best trial:
  Value: 0.6818121075630188
  Params: 
    learning_rate: 3.0819032743298975e-05
    per_device_train_batch_size: 8
    num_train_epochs: 3
